In [38]:
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib import animation

In [12]:
def allPerms(element):
    if len(element) <=1:
        yield element
    else:
        for perm in allPerms(element[1:]):
            for i in range(len(perm)+1):
                #nb str[0:1] works in both string and list contexts
                yield perm[:i] + element[0:1] + perm[i:]

def bruteForce(graph, s):
    cost = 1000
    path = []
    arr = list(range(1, 4))
    for allCities in list(allPerms(arr)):
        allCities = [0] + allCities + [0]
        currentCost = 0
        for i in range(1, len(allCities)):
            currentCost += graph[allCities[i - 1]][allCities[i]]
        if (currentCost < cost):
            cost = currentCost
            path = allCities
    path = [x + 1 for x in path]
    return [cost, path]

graph = [[0, 10, 15, 20], [10, 0, 35, 25],
        [15, 35, 0, 30], [20, 25, 30, 0]]
s = 0
print(bruteForce(graph, s))

[80, [1, 3, 4, 2, 1]]


In [41]:
def allCombinations(element, size):
    if (len(element) <= 1):
        yield element
    if (size == 0): 
        yield []
    else:
        for i in range(len(element) - size + 1):
            for combi in allCombinations(element[i + 1:], size - 1):
                yield [element[i]] + combi

def dynamicProgramming(G, s):
    n = len(G)
    C = [[np.inf for _ in range(1 << n)] for __ in range(n)]
    C[0][1] = 0
    for size in range(1, n):
        for S in allCombinations(list(range(1, n)), size):
            S = S + [0]
            print(S)
            k = sum([1 << _ for _ in S])
            for i in S:
                if (i == 0): continue
                index = k ^ (1 << i)
                for j in S:
                    if (j == i): continue
                    C[i][k] = min(C[i][k], C[j][index] + G[i][j])
    max_index = (1 << n) - 1
    return min([(C[i][max_index] + G[i][0], i) for i in range(n)])

# matrix representation of graph
graph = [[0, 10, 15, 20], [10, 0, 35, 25],
        [15, 35, 0, 30], [20, 25, 30, 0]]
s = 0
print(dynamicProgramming(graph, s))

[1, 0]
[2, 3, 0]
[2, 0]
[3, 0]
[3, 0]
[1, 2, 3, 0]
[1, 2, 0]
[1, 3, 0]
[1, 3, 0]
[2, 3, 0]
[2, 3, 0]
[2, 3, 0]
[1, 2, 3, 0]
[1, 2, 3, 0]
[1, 2, 3, 0]
(80, 1)


In [92]:
DNA_SIZE = 4            # DNA length
POP_SIZE = 100           # population size
MUTATION_RATE = 0.4   # mutation probability
N_GENERATIONS = 200

def getFitness(pop, G):
    fitness = np.zeros(POP_SIZE)
    for i in range(POP_SIZE):
        currentDNA = pop[i]
        fitness[i] = sum([G[currentDNA[j - 1]][currentDNA[j]] for j in range(1, DNA_SIZE + 1)])
    return fitness

def select(pop, fitness):    
    idx = np.random.choice(np.arange(POP_SIZE), size=POP_SIZE, replace=True,
                           p=fitness/fitness.sum())
    return pop[idx]

def mutate(parent):
    for point in range(1, DNA_SIZE - 1):
        if np.random.rand() < MUTATION_RATE:
            point1 = np.random.randint(1, DNA_SIZE)
            point2 = np.random.randint(1, DNA_SIZE)
            parent[point1], parent[point2] = parent[point2], parent[point1]
    return parent

def geneticAlgorithm(G, s):
    pop = np.array([[0] + list(np.random.permutation(range(1, DNA_SIZE))) + [0] for i in range(POP_SIZE)])
    for i in range(N_GENERATIONS):
        fitness = getFitness(pop, G)
        pop = select(pop, fitness)
        for parent in pop:
            child = mutate(parent)
            parent[:] = child
    fitness = getFitness(pop, G)
    elite = np.argmin(fitness)
    return (pop[elite][:], fitness[elite])
graph = [[0, 10, 15, 20], [10, 0, 35, 25],
        [15, 35, 0, 30], [20, 25, 30, 0]]
s = 0
shortestPath, cost = geneticAlgorithm(graph, s)
shortestPath = [i + 1 for i in shortestPath]
print(shortestPath, cost)

[1, 2, 4, 3, 1] 80.0


In [ ]:
%matplotlib auto

N_CITIES = 20  # DNA size
CROSS_RATE = 0.1
MUTATE_RATE = 0.02
POP_SIZE = 500
N_GENERATIONS = 500

class GA(object):
    def __init__(self, DNA_size, cross_rate, mutation_rate, pop_size, ):
        self.DNA_size = DNA_size
        self.cross_rate = cross_rate
        self.mutate_rate = mutation_rate
        self.pop_size = pop_size
        self.pop = np.vstack([np.random.permutation(DNA_size) for _ in range(pop_size)])

    def translateDNA(self, DNA, city_position):     # get cities' coord in order
        line_x = np.empty_like(DNA, dtype=np.float64)
        line_y = np.empty_like(DNA, dtype=np.float64)
        for i, d in enumerate(DNA):
            city_coord = city_position[d]
            line_x[i, :] = city_coord[:, 0]
            line_y[i, :] = city_coord[:, 1]
        line_x = np.c_[line_x, line_x[:, 0]]
        line_y = np.c_[line_y, line_y[:, 0]]
        return line_x, line_y

    def get_fitness(self, line_x, line_y):
        total_distance = np.empty((line_x.shape[0],), dtype=np.float64)
        for i, (xs, ys) in enumerate(zip(line_x, line_y)):
            total_distance[i] = np.sum(np.sqrt(np.square(np.diff(xs)) + np.square(np.diff(ys))))
        fitness = np.exp(self.DNA_size * 2 / total_distance)
        return fitness, total_distance

    def select(self, fitness):
        idx = np.random.choice(np.arange(self.pop_size), size=self.pop_size, replace=True, p=fitness / fitness.sum())
        return self.pop[idx]

    def crossover(self, parent, pop):
        if np.random.rand() < self.cross_rate:
            i_ = np.random.randint(0, self.pop_size, size=1)
            cross_points = np.random.randint(0, 2, self.DNA_size).astype(np.bool)
            keep_city = parent[~cross_points]
            swap_city = pop[i_, np.isin(pop[i_].ravel(), keep_city, invert=True)]
            parent[:] = np.concatenate((keep_city, swap_city))
        return parent

    def mutate(self, child):
        for point in range(1, self.DNA_size):
            if np.random.rand() < self.mutate_rate:
                swap_point = np.random.randint(1, self.DNA_size)
                swapA, swapB = child[point], child[swap_point]
                child[point], child[swap_point] = swapB, swapA
        return child

    def evolve(self, fitness):
        pop = self.select(fitness)
        pop_copy = pop.copy()
        for parent in pop:  # for every parent
            child = self.crossover(parent, pop_copy)
            child = self.mutate(child)
            parent[:] = child
        self.pop = pop


class TravelSalesPerson(object):
    def __init__(self, n_cities):
        self.city_position = np.random.rand(n_cities, 2)
        plt.ion()

    def plotting(self, lx, ly, total_d):
        plt.cla()
        plt.scatter(self.city_position[:, 0].T, self.city_position[:, 1].T, s=100, c='k')
        plt.plot(lx.T, ly.T, 'r-')
        plt.text(0.2, -0.1, "Total distance=%.2f" % total_d, fontdict={'size': 20, 'color': 'red'})
        plt.xlim((-0.1, 1.1))
        plt.ylim((-0.1, 1.1))
        plt.pause(0.01)

ga = GA(DNA_size=N_CITIES, cross_rate=CROSS_RATE, mutation_rate=MUTATE_RATE, pop_size=POP_SIZE)
env = TravelSalesPerson(N_CITIES)

for generation in range(N_GENERATIONS):
    lx, ly = ga.translateDNA(ga.pop, env.city_position)
    fitness, total_distance = ga.get_fitness(lx, ly)
    ga.evolve(fitness)
    best_idx = np.argmax(fitness)
    print('Gen:', generation, '| best fit: %.2f' % fitness[best_idx],)

    env.plotting(lx[best_idx], ly[best_idx], total_distance[best_idx])

plt.ioff()
plt.show()

Using matplotlib backend: Qt5Agg
Gen: 0 | best fit: 614.19
Gen: 1 | best fit: 614.19
Gen: 2 | best fit: 1430.15
Gen: 3 | best fit: 1430.15
Gen: 4 | best fit: 1825.17
Gen: 5 | best fit: 1825.17
Gen: 6 | best fit: 1991.74
Gen: 7 | best fit: 2528.98
Gen: 8 | best fit: 3652.61
Gen: 9 | best fit: 3091.32
Gen: 10 | best fit: 4748.15
Gen: 11 | best fit: 4748.15
Gen: 12 | best fit: 4748.15
Gen: 13 | best fit: 5667.18
Gen: 14 | best fit: 5667.18
Gen: 15 | best fit: 7888.20
Gen: 16 | best fit: 7888.20
Gen: 17 | best fit: 7888.20
Gen: 18 | best fit: 7888.20
Gen: 19 | best fit: 9041.81
Gen: 20 | best fit: 9041.81
Gen: 21 | best fit: 12909.41
Gen: 22 | best fit: 12909.41
Gen: 23 | best fit: 12909.41
Gen: 24 | best fit: 12909.41
Gen: 25 | best fit: 15872.63
Gen: 26 | best fit: 15872.63
Gen: 27 | best fit: 15872.63
Gen: 28 | best fit: 20383.78
Gen: 29 | best fit: 20383.78
Gen: 30 | best fit: 20383.78
Gen: 31 | best fit: 28573.87
Gen: 32 | best fit: 29094.79
Gen: 33 | best fit: 37138.62
Gen: 34 | best